In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'
---require 'Adversarials.exploitability'
require 'Adversarials.exploitabilityVS'



In [2]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{200, 200}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
--- exploitability
exploit = TreeValues()

In [3]:
--- Initialize trees
tree1 = builder:build_tree(params)
tree2 = builder:build_tree(params)

In [4]:
solver2 = TreeCFR()
solver2:run_cfr(tree2,starting_ranges,100)

In [5]:
--- NNWarm Starting
dataNN = TreeData(tree2)
dataNN:get_training_set(tree2,1)

In [6]:
tree_eval = StrategyEvaluator(tree1,tree2)

In [7]:
local pot_table = {}
local size_sample = 50
for i = 1,50 do
    tree_eval:play_several_games(tree1,tree2,size_sample)
    table.insert(pot_table,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table):mean())

In [8]:
cards = {}
cards.P1 = 3
cards.P2 = 4
cards.board = 1

player_2 won: 	-14.56


In [308]:
pot_table1 = {}
local size_sample = 1000
for i = 1,2 do
    tree_eval:play_deterministic_games(tree1,tree2,size_sample,cards)
    table.insert(pot_table1,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

player_2 won: 	4.5999999046326
player_2 std: 	14.990663626285


In [28]:
cardsInv = {}
cardsInv.P1 = cards.P2
cardsInv.P2 = cards.P1
cardsInv.board = cards.board

In [307]:
pot_table2 = {}
local size_sample = 500
for i = 1,2 do
    tree_eval:play_deterministic_games(tree1,tree2,size_sample,cardsInv)
    table.insert(pot_table2,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table2):std())

player_2 won: 	-5.7999999523163
player_2 std: 	2.5455843448366


In [30]:
---plot = Plot():histogram(torch.FloatTensor(pot_table1)):draw()

In [31]:
---plot = Plot():histogram(torch.FloatTensor(pot_table2)):draw()

In [304]:
function get_first_mask_tensor(index)
    local output = torch.range(1,6)
    if index ==1 then
        return torch.range(2,6)
    end
    local out1 = output:clone():narrow(1,1,index-1)
    local out2 = output:clone():narrow(1,index+1,6-index)
    local result = out1:cat(out2)
    return result
end

function get_second_mask_tensor(input_tensor,index)
    local result = {}
    local input = input_tensor:clone()
    for i = 1,input:size(1) do
        if input[i] ~= index then
            table.insert(result,input[i])
        end
    end
    return torch.Tensor(result)
end


function construct_possible_game_states()
    local states_tensor = torch.Tensor()
    local first_card_tensor = torch.Tensor({1,3,5})
    for i = 1,first_card_tensor:size(1) do
        local second_card_tensor = get_first_mask_tensor(first_card_tensor[i])
        for j =1, second_card_tensor:size(1) do
            local third_card_tensor = get_second_mask_tensor(second_card_tensor,second_card_tensor[j])
            for k = 1,third_card_tensor:size(1) do
                local states = torch.Tensor({first_card_tensor[i],second_card_tensor[j],third_card_tensor[k]}):view(1,3)
                states_tensor = states_tensor:cat(states,1)
            end
        end
    end
    return states_tensor
end
    


In [305]:
x = construct_possible_game_states()

In [299]:
 f = get_first_mask_tensor(1)

In [300]:
f

 2
 3
 4
 5
 6
[torch.FloatTensor of size 5]



In [278]:
f1= get_second_mask_tensor(f,3)

In [279]:
f1

 1
 2
 4
 5
 6
[torch.FloatTensor of size 5]

